In [ ]:
import numpy as np
import cv2
from tqdm.auto import tqdm

In [ ]:
def read_video(path: str) -> np.array:
    cap = cv2.VideoCapture(path)
    frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

    frames = []
    for _ in tqdm(range(frame_count), desc='Reading video'):
        ret, frame = cap.read()
        if not ret:
            break

        # converting to grayscale for simplicity
        frames.append(cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY))

    cap.release()
    frames = np.array(frames)

    return frames

In [ ]:
# video_path = '/Volumes/Patrick/Proband06/Logitech HD Pro Webcam C920.avi'
video_path = '/Users/patrick/Desktop/Proband06/Logitech HD Pro Webcam C920.avi'
video_frames = read_video(video_path)

In [ ]:
# Chest region
# pixel_x, pixel_y = 400, 400
pixel_x, pixel_y = 400, 375

# Headshot
# pixel_x, pixel_y = 180, 350

# Wall
# pixel_x, pixel_y = 100, 100

In [ ]:
import matplotlib.pyplot as plt

# Draw a point on the pixel in the first frame
frame = video_frames[0].copy()

plt.imshow(frame, cmap='gray')
plt.scatter(pixel_y, pixel_x, color='red')
plt.show()

In [ ]:
pixels = []

for frame in video_frames:
    pixels.append(frame[pixel_x, pixel_y])

In [ ]:
# Plot the pixel intensity over time in an interactive plot
import plotly.express as px

fig = px.line(y=pixels, title='Pixel intensity over time')
fig.show()

In [ ]:
import numpy as np
from scipy.fft import fft

# Normalize the pixel values
pixels_normalized = np.array(pixels) / 255

# Calculate the fast fourier transform of the thorax abdomen data
pixels_fft = fft(pixels_normalized)

# Calculate the frequency
freq = np.fft.fftfreq(len(pixels_fft), 1 / 30)

# Remove the negative frequencies
pixels_fft = pixels_fft[freq > 0]
freq = freq[freq > 0]

In [ ]:
# Plot the frequency spectrum
fig = px.line(x=freq, y=np.abs(pixels_fft), title='Frequency spectrum')
fig.show()

In [ ]:
# Find the frequency with the highest amplitude
max_freq = freq[np.argmax(np.abs(pixels_fft))]

# Calculate the breathing rate in beats per minute
breathing_rate = max_freq * 60

max_freq, breathing_rate

In [ ]:
# Only keep the frequencies between 0.1 and 0.6 Hz
bandpass_filter = (freq > 0.1) & (freq < 0.6)

# Remove the negative frequencies
filtered_fft = pixels_fft[bandpass_filter]
filtered_freq = freq[bandpass_filter]

In [ ]:
# Plot the filtered frequency spectrum
fig = px.line(x=filtered_freq, y=np.abs(filtered_fft), title='Filtered Frequency spectrum')
fig.show()

In [ ]:
# Find the frequency with the highest amplitude
max_freq = filtered_freq[np.argmax(np.abs(filtered_fft))]

# Calculate the breathing rate in beats per minute
breathing_rate = max_freq * 60

max_freq, breathing_rate